# TP3 - Views, Updates and Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP2.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [2]:
import sqlite3

In [3]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [5]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
CLIENT:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PRODUCTION:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
PRODUCTOR:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
RBA:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
REGION_LIEU:
  0: LIEU(TEXT)
  1: REGION(TEXT)
SALES:
  0: NB(NUM)
  1: NV(NUM)
  2: QTE(NUM)
  3: LIEU(TEXT)
  4: DATES(NUM)
WINE:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
WINE2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


Again, we will use **`%%sql`** magic for our queries

In [6]:
%load_ext sql
%sql sqlite:///wine.db

u'Connected: None@wine.db'

Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP2

In [7]:
%%sql 
-- Create PRODUCTOR TABLE 
DROP TABLE IF EXISTS PRODUCTOR;
CREATE TABLE PRODUCTOR AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1
WHERE NP IS NOT NULL; 

-- Create WINE TABLE 
DROP TABLE IF EXISTS WINE;
CREATE TABLE WINE AS
SELECT  DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL; 

--  Create PRODUCTION TABLE 
DROP TABLE IF EXISTS PRODUCTION;
CREATE TABLE PRODUCTION AS
SELECT   NP,NV, QTE
FROM MASTER1
WHERE NV IS NOT NULL or NP is not null; 
-- Create PRODUCTION TABLE 
DROP TABLE IF EXISTS PRODUCTION;
CREATE TABLE PRODUCTION AS
SELECT   NP,NV, QTE
FROM MASTER1
WHERE NV IS NOT NULL or NP is not null; 

-- Create WINE2 TABLE 
DROP TABLE IF EXISTS WINE2;
CREATE TABLE WINE2 AS
SELECT  DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2
WHERE NV IS NOT NULL; 
-- Create CLIENT TABLE 
DROP TABLE IF EXISTS CLIENT;
CREATE TABLE CLIENT AS
SELECT  DISTINCT NB, NOM, PRENOM,TYPE
FROM MASTER2
WHERE NB IS NOT NULL; 
--- Create SALES TABLE 
DROP TABLE IF EXISTS SALES;
CREATE TABLE SALES AS
SELECT  NB,NV, QTE, LIEU,DATES
FROM MASTER2
WHERE NV IS NOT NULL or NB is not null; 
--- Create REGION_LIEU TABLE 
DROP TABLE IF EXISTS REGION_LIEU;
CREATE TABLE REGION_LIEU AS
SELECT  DISTINCT LIEU, REGION
FROM MASTER2
WHERE NV IS NOT NULL or NB is not null; 


Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [8]:
%%sql
DROP VIEW IF EXISTS bons_buveurs;
CREATE VIEW bons_buveurs AS
SELECT NB,NOM, PRENOM, TYPE
FROM CLIENT
WHERE TYPE='gros' or TYPE='moyen'


Done.
Done.


[]

In [7]:
# Test
%%sql SELECT * FROM bons_buveurs ORDER BY nb;

Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [9]:
%%sql 
DROP VIEW IF EXISTS buveurs_asec;
CREATE VIEW buveurs_asec AS
select S.nb, C.nom, C.prenom, C.type
from SALES S , CLIENT C
where S.NB=C.NB
GROUP BY S.NB
HAVING count(distinct nv)=0


Done.
Done.


[]

In [9]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [10]:
%%sql 
DROP VIEW IF EXISTS buveurs_achats;
CREATE VIEW buveurs_achats AS
select S.nb, C.nom, C.prenom, C.type
from SALES S , CLIENT C
where S.NB=C.NB
GROUP BY S.NB
HAVING count(distinct nv)>=1

Done.
Done.


[]

In [11]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

Done.


NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,petit
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
9,Ajar,Emile,petit
10,Andersen,Yann,gros


#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [11]:
%%sql 
DROP VIEW IF EXISTS q83pl;
CREATE VIEW q83pl AS
select RL.LIEU, CRU , SUM(QTE) as QTE_BUE
from REGION_LIEU RL, SALES S, WINE2 W
where RL.LIEU=S.LIEU and
       S.NV=W.NV and 
       strftime('%Y', DATES)='1983'
GROUP BY RL.LIEU, CRU 

Done.
Done.


[]

In [74]:
%%sql 
SELECT * 
FROM q83pl 
limit 10 

Done.


LIEU,CRU,QTE_BUE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


In [13]:
# Test
%sql SELECT * FROM q83pl;

Done.


LIEU,CRU,QTE_BUE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

In [12]:
%%sql 
DROP VIEW IF EXISTS q83pl_qte_asc;
CREATE VIEW q83pl_qte_asc AS
select RL.LIEU, CRU , SUM(QTE) as QTE_BUE
from REGION_LIEU RL, SALES S, WINE2 W
where RL.LIEU=S.LIEU and
       S.NV=W.NV and 
       strftime('%Y', DATES)='1983'
GROUP BY RL.LIEU, CRU 
ORDER BY QTE_BUE 

Done.
Done.


[]

In [59]:
%%sql 
select * 
from q83pl_qte_asc 
limit 15

Done.


LIEU,CRU,QTE_BUE
PARIS,Coteaux du Tricastin,1
RENNES,Mercurey,1
LYON,Julienas,2
CAEN,Seyssel,3
PARIS,Pouilly Vinzelles,3
PARIS,Beaujolais Primeur,4
LILLE,Pommard,5
LYON,Beaujolais Villages,10
ROCQUENCOURT,Saint Amour,80
ROCQUENCOURT,Beaujolais Villages,260


___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [13]:
%%sql 
-- Create PRODUCTOR RBB 
DROP TABLE IF EXISTS RBB;
CREATE TABLE RBB AS
SELECT *
FROM bons_buveurs
ORDER BY NB


Done.
Done.


[]

In [14]:
# Test
%sql SELECT * FROM RBB;

Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may differ from the one in your table)

In [15]:
%%sql
select SALES.NB,NOM,PRENOM, TYPE, sum(QTE) as total
from SALES , CLIENT 
WHERE SALES.NB=CLIENT.NB AND TYPE!='gros'
GROUP BY SALES.NB 
HAVING  total>100


Done.


NB,NOM,PRENOM,TYPE,total
2,Artaud,Antonin,moyen,583
5,Audiberti,Jacques,petit,113
9,Ajar,Emile,petit,140
44,Gide,Andre,petit,171


Update instances

In [16]:
%%sql
UPDATE ClIENT
SET TYPE = 'gros' 
WHERE NB in (select SALES.NB
             from SALES , CLIENT 
              WHERE SALES.NB=CLIENT.NB AND TYPE!='gros'
              GROUP BY SALES.NB 
               HAVING  sum(qte)>100
              )

4 rows affected.


[]

4 rows affected.


[]

#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

Answer: 
The table RBB is the same , while the view bons_buveurs have more rows than before. 
RBB is a **table** in complete independence from the original table CLIENT, they don't have neither dependencies nor referential integrity constraints , for these reasons the content of table RBB don't change after the update. Wheras bons_buveurs is a "permanent" **VIEW** build on the table CLIENT. This is why updating the the table CLIEN may cause updates also in the VIEW. In this case new rows were added to bons_buveurs because after the update, there are new rows included into the result of the query used to buid bons_buveurs. 

#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [17]:
%%sql 
-- Create RBA 
DROP TABLE IF EXISTS RBA;
CREATE TABLE RBA AS
SELECT *
FROM buveurs_asec
ORDER BY NB


Done.
Done.


[]

In [18]:
%%sql 
SELECT * 
FROM RBA
LIMIT 5

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen


In [74]:
# Test
%sql SELECT * FROM RBA

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [75]:
%%sql 
INSERT INTO RBA (NB,NOM,PRENOM,TYPE)
VALUES (101, 'Fotso', 'Axel', 'petit')

1 rows affected.


[]

In [76]:
# Test
%sql SELECT * FROM RBA

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain

Answer: The table **RBA** contains the row just inserted while the view **buveurs_asec** is the same as before (no change ) . 
**Explanation** : **RBA** is a table built querying the view **buveurs_asec** , afther this operation , the table is not longer linked to the the view , So any update on the table RBA don't affect the view which can be affected only if its "building query" gives a different result.

___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

Answer:  Our database contains 4 tables (the primary keys are highlighted in **bold** : 
- USER (**Uid**, Name, NickName, Email, Date_of_Birth, Address ) : This is the table containing basic informations about users. Each tuple is uniquely identified by the user id Uid. 
  - Uid is a number representing the user id 
  - Name, Nickname, Email are VARCHAR 
  - Date_of_birth is a date 
  - Address is a -structured- VARCHAR in the form : street-City-Sate-Country, PC . This string is       generated from the web site. Some user defined function can help to split it whith the objective of extracting only the field of the adress we are interessed in. 
- RELATIONSHIP(**User1** ,**User2**) : This table medels the relationships between the users . If a tuple (U1, U2) exits then it means that the user with id U2 is friend of user with id U2. Obviously this implies (for the sake of reflexivity) that also the tuple (U2,U1) exists. This reflexibility constraint can be managed wiht a trigger on the table RELATION. 
- POST (**postId**, UserId, date, text, attachements, isRepost,originalId ) : This table models the posts. Each post is identified with a postId . One post is associated to only one user and one user can be owner of more than one post. the column isRepost is a boolean , when set to 1 this means that the post is a repost of the post wiht id equal to originalId ( nullable ). The field attachement can be a link to repository (on an internal server) containing images , sounds and video.  

  

#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.

Answer: We suppose we want to display the posts to user with id=1541
```sql 
-- This query is selecting text and attachment of at most 15 posts posted by users that are friend of user whit id= 1541
CREATE VIEW loginvisual  as
SELECT  text, attachemnts 
FROM POST
WHERE UserId in (Select User2 
                 FROM RELATIONSHIP 
                 WHERE User2=1541)
LIMIT 15
```  
This way of displaying information as said is too simplistic , this is why in oder to improve this i would add to each post a **DegreeOfInterest** which is a number computed based on the number of friend that rated (liked) / commented (these features should be added too) the post and the total number of users in the entire social network that (liked) / commented (these features should be added too) the post. 
